<a href="https://colab.research.google.com/github/Aakash7khadka/MachineLearning/blob/master/Share_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import requests
import re
import concurrent.futures
import pandas as pd

In [48]:
from lxml import html

In [49]:
url="https://www.nepalipaisa.com/Listed-Companies.aspx"
response=requests.get(url)

In [50]:
tree=html.fromstring(response.content)

In [51]:
xpath_list_of_companies='//*[(@id = "tblListedCompanies")]//a'

In [60]:
list=[]
for x in tree.xpath(xpath_list_of_companies):
  list.append(x.text)


In [69]:
re.findall("([a-zA-z0-9]+)",list[0])[-1]

'NMBD2085'

In [92]:
final_list=[]
for x in list:
  final_list.append(re.findall("([a-zA-z0-9]+)",x)[-1] )

In [123]:
final_list[0]

'NMBD2085'

In [214]:

list_of_lists=[]

In [215]:
%%time
def transform(symbol):

  data={}
  data['symbol']=symbol

  url="https://merolagani.com/CompanyDetail.aspx?symbol="+symbol
  response=requests.get(url)
  tree=html.fromstring(response.content)
  for i in range(1,13):
    key=tree.xpath(f'//*[@id="accordion"]/tbody[{i}]/tr/th')
    element=tree.xpath(f'//*[@id="accordion"]/tbody[{i}]/tr/td')
    data[key[0].text.strip()]=element[0].text.strip()
    data['Market Price']=tree.xpath('//*[(@id = "ctl00_ContentPlaceHolder1_CompanyDetail1_lblMarketPrice")]')[0].text.strip()
    data['% Change']=tree.xpath('//*[(@id = "ctl00_ContentPlaceHolder1_CompanyDetail1_lblChange")]')[0].text.strip()
    data['1 Year Yield']=tree.xpath('//*[(@id = "ctl00_ContentPlaceHolder1_CompanyDetail1_lblYearYeild")]')[0].text.strip()
  # lists=[x for x in lists if x]
  list_of_lists.append(data) 



CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 14.1 µs


In [216]:
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
  executor.map(transform,final_list)
  executor.shutdown(wait=True)

In [233]:
df=pd.DataFrame(list_of_lists)
df1=df.iloc[:,7].str.split('-',expand=True)
df1.columns=['52weekhigh','52weeklow']
pd.concat([df,df1],axis=1)

,symbol,Sector,Market Price,% Change,1 Year Yield,Shares Outstanding,Last Traded On,52 Weeks Low - High,180 Day Average,120 Day Average,EPS,P/E Ratio,Book Value,52weekhigh,52weeklow
0,ICFCD83,Corporate Debenture,"1,162.00",-1.53 %,0.00%,,2021/04/15 03:00:00,"1,202.00-1,086.00","1,163.83","1,163.83",0.00,0.00,0.00,"1,202.00","1,086.00"
1,SBIBD86,Corporate Debenture,"1,046.00",0 %,0.00%,,2021/04/15 03:00:00,"1,071.00-1,000.00","1,044.52","1,048.54",0.00,0.00,0.00,"1,071.00","1,000.00"
2,AVU,Manufacturing And Processing,58.00,0 %,0.00%,,2006/02/09 03:00:00,0.00-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,CIT,Investment,"4,159.00",0.19 %,70.45%,"29,999,310.00",2021/04/15 03:00:00,"5,480.00-2,070.00","4,303.56","4,269.91",21.51,193.35,896.96,"5,480.00","2,070.00"
4,PBLD86,Corporate Debenture,0.00,0 %,0.00%,,,0.00-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,VLBSPO,Promotor Share,571.00,1.96 %,0.00%,,2020/12/14 03:00:00,571.00-571.00,571.00,0.00,0.00,0.00,0.00,571.00,571.00
438,WOMI,Microfinance,"1,796.00",-0.5 %,112.29%,"3,612,020.00",2021/04/15 03:00:00,"1,824.00-745.00","1,389.63","1,389.63",29.42,61.05,140.59,"1,824.00",745.00
439,YHL,Hotels And Tourism,600.00,0 %,0.00%,,2005/12/19 03:00:00,0.00-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
440,WMBFPO,Promotor Share,101.00,0 %,0.00%,,2017/12/24 03:00:00,101.00-101.00,101.00,0.00,0.00,0.00,0.00,101.00,101.00


In [209]:
# data={}
# url="https://merolagani.com/CompanyDetail.aspx?symbol=upper"
# response=requests.get(url)
# tree=html.fromstring(response.content)
# for i in range(1,13):
#   key=tree.xpath(f'//*[@id="accordion"]/tbody[{i}]/tr/th')
#   element=tree.xpath(f'//*[@id="accordion"]/tbody[{i}]/tr/td')
 

#   print(key[0].text.strip(),element[0].text.strip())
  
#   # print(key[0].text.strip(),element[0].text.strip())
#   data[key[0].text.strip()]=element[0].text.strip()
#   data['Market Price']=tree.xpath('//*[(@id = "ctl00_ContentPlaceHolder1_CompanyDetail1_lblMarketPrice")]')[0].text.strip()
#   data['% Change']=tree.xpath('//*[(@id = "ctl00_ContentPlaceHolder1_CompanyDetail1_lblChange")]')[0].text.strip()
#   data['1 Year Yield']=tree.xpath('//*[(@id = "ctl00_ContentPlaceHolder1_CompanyDetail1_lblYearYeild")]')[0].text.strip()
# data


Sector Hydro Power
Shares Outstanding 105,900,000.00
Market Price 
% Change 
Last Traded On 2021/04/15 03:00:00
52 Weeks Low - High 908.00-215.00
180 Day Average 496.47
120 Day Average 568.12
1 Year Yield 
EPS -0.60
P/E Ratio -1,458.33
Book Value 92.24


{'% Change': '-0.79 %',
 '1 Year Yield': '285.46%',
 '120 Day Average': '568.12',
 '180 Day Average': '496.47',
 '52 Weeks Low - High': '908.00-215.00',
 'Book Value': '92.24',
 'EPS': '-0.60',
 'Last Traded On': '2021/04/15 03:00:00',
 'Market Price': '875.00',
 'P/E Ratio': '-1,458.33',
 'Sector': 'Hydro Power',
 'Shares Outstanding': '105,900,000.00'}

In [239]:
df.to_csv()

',symbol,Sector,Market Price,% Change,1 Year Yield,Shares Outstanding,Last Traded On,52 Weeks Low - High,180 Day Average,120 Day Average,EPS,P/E Ratio,Book Value\n0,ICFCD83,Corporate Debenture,"1,162.00",-1.53 %,0.00%,,2021/04/15 03:00:00,"1,202.00-1,086.00","1,163.83","1,163.83",0.00,0.00,0.00\n1,SBIBD86,Corporate Debenture,"1,046.00",0 %,0.00%,,2021/04/15 03:00:00,"1,071.00-1,000.00","1,044.52","1,048.54",0.00,0.00,0.00\n2,AVU,Manufacturing And Processing,58.00,0 %,0.00%,,2006/02/09 03:00:00,0.00-0.00,0.00,0.00,0.00,0.00,0.00\n3,CIT,Investment,"4,159.00",0.19 %,70.45%,"29,999,310.00",2021/04/15 03:00:00,"5,480.00-2,070.00","4,303.56","4,269.91",21.51,193.35,896.96\n4,PBLD86,Corporate Debenture,0.00,0 %,0.00%,,,0.00-0.00,0.00,0.00,0.00,0.00,0.00\n5,NBBD2085,Corporate Debenture,"1,046.00",0.1 %,0.00%,,2021/04/15 03:00:00,"1,065.00-1,045.00","1,054.00","1,054.00",0.00,0.00,0.00\n6,CCBLPO,Promotor Share,112.00,0 %,-1.75%,,2021/04/15 03:00:00,114.00-112.00,112.00,112.00,0.00,0.00,0.00\n7,

In [247]:
df.set_index('symbol').to_csv()

'symbol,Sector,Market Price,% Change,1 Year Yield,Shares Outstanding,Last Traded On,52 Weeks Low - High,180 Day Average,120 Day Average,EPS,P/E Ratio,Book Value\nICFCD83,Corporate Debenture,"1,162.00",-1.53 %,0.00%,,2021/04/15 03:00:00,"1,202.00-1,086.00","1,163.83","1,163.83",0.00,0.00,0.00\nSBIBD86,Corporate Debenture,"1,046.00",0 %,0.00%,,2021/04/15 03:00:00,"1,071.00-1,000.00","1,044.52","1,048.54",0.00,0.00,0.00\nAVU,Manufacturing And Processing,58.00,0 %,0.00%,,2006/02/09 03:00:00,0.00-0.00,0.00,0.00,0.00,0.00,0.00\nCIT,Investment,"4,159.00",0.19 %,70.45%,"29,999,310.00",2021/04/15 03:00:00,"5,480.00-2,070.00","4,303.56","4,269.91",21.51,193.35,896.96\nPBLD86,Corporate Debenture,0.00,0 %,0.00%,,,0.00-0.00,0.00,0.00,0.00,0.00,0.00\nNBBD2085,Corporate Debenture,"1,046.00",0.1 %,0.00%,,2021/04/15 03:00:00,"1,065.00-1,045.00","1,054.00","1,054.00",0.00,0.00,0.00\nCCBLPO,Promotor Share,112.00,0 %,-1.75%,,2021/04/15 03:00:00,114.00-112.00,112.00,112.00,0.00,0.00,0.00\nCMB,Finance,140.0